Table of Contents:
* [1. Introduction](#Introduction)
* [2. Foods by Nutrient](#Foods-by-Nutrient)
* [3. Nutrients by Food](#Nutrients-by-Food)

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import difflib

In [ ]:
import re

In [ ]:
food = pd.read_csv('../input/food-and-nutrient-data/food.csv')

In [ ]:
food_nutrient = pd.read_csv('../input/food-and-nutrient-data/food_nutrient.csv')

In [ ]:
nutrient = pd.read_csv('../input/food-and-nutrient-data/nutrient.csv')

# Introduction

In [ ]:
food.head()

In [ ]:
food.info()

In [ ]:
food['description'] = food['description'].astype(str)

In [ ]:
f'number of unique food items {pd.unique(food.iloc[:, 2]).size}'

In [ ]:
food_nutrient.head()

In [ ]:
food_nutrient.info()

In [ ]:
food_nutrient['footnote'].unique()

In [ ]:
food_nutrient[food_nutrient['footnote']=='Trace amount']

In [ ]:
f'number of unique nutrients {pd.unique(food_nutrient.iloc[:, 2]).size}'

In [ ]:
nutrient.head()

In [ ]:
nutrient.info()

In [ ]:
plt.figure(figsize=(15, 5))
sns.countplot(x='unit_name', data=nutrient, order=nutrient['unit_name'].value_counts().index)

Merging with nutrient dataframe for names of nutrients.

In [ ]:
nutrients_with_names = nutrient.merge(food_nutrient.drop('id', axis=1), left_on='id', right_on='nutrient_id')

In [ ]:
nutrients_with_names.head()

In [ ]:
nutrients_with_names.shape

In [ ]:
nutrients_with_names.isnull().sum()

In [ ]:
nutrients_with_names = nutrients_with_names[['nutrient_id', 'name', 'amount', 'unit_name', 'fdc_id']]

merging food and nutrients_with_names to make combined dataframe

In [ ]:
df = pd.merge(food, nutrients_with_names, on='fdc_id')
df

# Foods by Nutrient

Most mentioned food items

In [ ]:
nutrients = pd.unique(df['name'])
sorted(nutrients)

In [ ]:
f'number of unique nutrient names {nutrients.size}'

In [ ]:
vitamins = ['Vitamin A, RAE', 'Thiamin', 'Riboflavin', 'Niacin', 'Pantothenic acid', 'Vitamin B-6', 'Biotin', 'Folate, total', 'Vitamin B-12',
 'Vitamin C, total ascorbic acid', 'Vitamin D (D2 + D3)', 'Vitamin D2 (ergocalciferol)', 'Vitamin D3 (cholecalciferol)', 'Vitamin E (alpha-tocopherol)', 
 'Vitamin K (Dihydrophylloquinone)', 'Vitamin K (Menaquinone-4)', 'Vitamin K (phylloquinone)']

In [ ]:
minerals = ['Potassium, K', 'Sodium, Na', 'Calcium, Ca', 'Phosphorus, P', 'Magnesium, Mg', 'Iron, Fe', 'Zinc, Zn', 'Manganese, Mn',
             'Copper, Cu', 'Iodine, I', 'Selenium, Se', 'Molybdenum, Mo', 'Cobalt, Co', 'Nickel, Ni', 'Boron, B']

measurement unit of vitamins

In [ ]:
for i in vitamins:
    print(df[df['name']==i].iloc[0]['unit_name'], 'is unit of measurement for', i )

measurement unit of minerals

In [ ]:
for i in minerals:
    print(df[df['name']==i].iloc[0]['unit_name'], 'is unit of measurement for', i )

In [ ]:
df.groupby('description').count().sort_values(by='amount', ascending=False).head(10)

Most mentioned nutrients

In [ ]:
df.groupby(['name']).count().sort_values(by='amount', ascending=False).head(10)

In [ ]:
matches = difflib.get_close_matches('vitamin D', nutrients, n=15, cutoff=.2)
matches

In [ ]:
r = re.compile("Choline")

In [ ]:
set(filter(r.match, nutrients))

In [ ]:
df[df['name']=='Calcium, Ca'].sort_values(by='amount', ascending=False).head(10)

## Finding by Modifying Description

When we sort the foods by amount of protein they have, there are many similar food items near to each other. To list different foods, we can split the description and remove duplicates

In [ ]:
def filter_description(df, unique_words_in_description):
    return df.loc[(df['description'].str.split(r",|;", expand=True)).drop_duplicates(subset=[i for i in range(unique_words_in_description)]).index]

In [ ]:
filter_description(df[df['name']=='Vitamin K (phylloquinone)'].sort_values(by='amount', ascending=False), 1).head(10)

# Nutrients by Food

## making wide dataframe

Removing duplicated nutrient names

In [ ]:
nutrient['name'][nutrient['name'].duplicated()]

In [ ]:
nutrient[nutrient['name']=='Energy']

In [ ]:
(df[(df['name']=='Energy') & (df['unit_name']=='KCAL')].reset_index()['fdc_id'] == df[(df['name']=='Energy') & (df['unit_name']=='kJ')].reset_index()['fdc_id']).all()

In [ ]:
df[df['nutrient_id']==1062].index

In [ ]:
df_1 = df.drop(df[df['nutrient_id']==1062].index)
df_1

In [ ]:
df_1.columns

In [ ]:
df_1.drop(['data_type', 'food_category_id','publication_date'], axis=1, inplace=True)

Converting nutrients which are listed in rows to columns

In [ ]:
wide_df = df_1.set_index(['fdc_id', 'description', 'name'])['amount'].unstack().reset_index().rename_axis(None, axis=1)
wide_df

Number of food items with vitamins

In [ ]:
wide_df.count()[vitamins]

In [ ]:
wide_df.count()[vitamins].plot(kind='bar', figsize=(10,5))

Number of food items with minerals

In [ ]:
wide_df.count()[minerals]

In [ ]:
wide_df.count()[minerals].plot(kind='bar', figsize=(10,5))

sum of rankings of foods by vitamin content

In [ ]:
wide_df.sort_values(by='Vitamin B-6', ascending=False)[['description', 'Vitamin B-6']].head(10)

foods with high protein to carbohydrate value

In [ ]:
wide_df.loc[(wide_df['Protein']/wide_df['Carbohydrate, by summation']).sort_values(ascending=False).index] \
    [['description', 'Protein', 'Carbohydrate, by summation']].head(10)

In [ ]:
matches = difflib.get_close_matches('Egg', list(df['description']), n=15, cutoff=.5)
matches

In [ ]:
r = re.compile('Egg')

In [ ]:
set(filter(r.match, list(df['description'])))

## high vitamins and minerals

Recommended Dietary Allowance (RDA): average daily level of intake sufficient to meet the nutrient requirements of nearly all (97%-98%) healthy people taken from from https://ods.od.nih.gov/HealthInformation/Dietary_Reference_Intakes.aspx

In [ ]:
minerals_rda = dict(zip(minerals[:12], [3400, 1500, 1000, 700, 400, 18, 11, 2.3, .9, 150, 55, 45]))

foods by their mineral content as a percentage of Recommended Dietary Allowance

In [ ]:
wide_df_minerals = wide_df[['description']].join(wide_df.loc[:, minerals_rda.keys()] / minerals_rda.values() * 100)
wide_df_minerals

In [ ]:
wide_df_minerals.loc[wide_df_minerals.loc[:,minerals_rda.keys()].sum(axis=1).sort_values(ascending=False).index].head(10)

In [ ]:
wide_df['description'].str.contains('Salt').tail(10)

In [ ]:
wide_df_minerals.loc[wide_df_minerals[~wide_df['description'].str.contains('Salt')].loc[:,minerals_rda.keys()].sum(axis=1).sort_values(ascending=False).index].round(2).head(10)

In [ ]:
vitamins_rda = dict(zip(vitamins[:10] + vitamins[12:14] + vitamins[16:], [900, 1.2, 1.3, 16, 5, 1.3, 30, 400, 2.4, 90, 15, 15, 120]))

foods by their vitamin content as a percentage of Recommended Dietary Allowance

In [ ]:
wide_df_vitamins = wide_df[['description']].join(wide_df.loc[:, vitamins_rda.keys()] / vitamins_rda.values() * 100)
wide_df_vitamins

In [ ]:
wide_df_vitamins.loc[wide_df_vitamins.loc[:,vitamins_rda.keys()].sum(axis=1).sort_values(ascending=False).index].round(2).head(10)

Filtering description to view different food items

In [ ]:
def filter_description(df, unique_words_in_description):
    return df.loc[(df['description'].str.split(r",|;|-", expand=True)).drop_duplicates(subset=[i for i in range(unique_words_in_description)]).index]

In [ ]:
filter_description(wide_df_minerals.loc[wide_df_minerals.loc[:,minerals_rda.keys()].sum(axis=1).sort_values(ascending=False).index], 1).head(10)

In [ ]:
filter_description(wide_df_vitamins.loc[wide_df_vitamins.loc[:,vitamins_rda.keys()].sum(axis=1).sort_values(ascending=False).index], 1).head(10)

In [ ]:
wide_df_minerals[wide_df_minerals['description'].isin(set(filter(r.match, list(df['description'].values))))][['description'] + list(minerals_rda.keys())]